In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sss').getOrCreate()




In [0]:
dbutils.fs.cp("/FileStore/AWS_CNF/dwh.cfg", "file:///tmp/dwh.cfg") 
import configparser
config = configparser.ConfigParser()
config.read_file(open('/tmp/dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

In [0]:
ACCESS_KEY = KEY
# Encode the Secret Key to remove any "/" characters
SECRET_KEY = SECRET.replace("/", "%2F")
AWS_BUCKET_NAME = "s3pysparkfiledump/databricks_file"
MOUNT_NAME = "/mnt/Ashish"

In [0]:
try:
  dbutils.fs.mount("s3a://{}:{}@{}".format(ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME), MOUNT_NAME)
except:
  print("""{} already mounted. Unmount using `dbutils.fs.unmount("{}")` to unmount first""".format(MOUNT_NAME, MOUNT_NAME))

/mnt/Ashish already mounted. Unmount using `dbutils.fs.unmount("/mnt/Ashish")` to unmount first

In [0]:
#from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType
#csv_schema = StructType([ \
#    StructField("car",StringType(),True), \
#    StructField("price",FloatType(),True), \
#    StructField("body",StringType(),True), \
#    StructField("mileage", IntegerType(), True), \
#    StructField("engV", StringType(), True),\
#    StructField("engType", StringType(), True),\
#    StructField("registration", StringType(), True),\
#    StructField("year", IntegerType(), True),\
#    StructField("model", StringType(), True),\
#    StructField("drive", StringType(), True),\
   
 # ])

In [0]:
#car_stream_data = spark.readStream \
#                      .format("cloudFiles")\
#                      .option("cloudFiles.format","csv")\
#                      .schema(csv_schema)\
#                      .load("dbfs:/mnt/Ashish/*")
#car_stream_data.display()

Other Option to read the strem

In [0]:
car_stream_data = spark.readStream.format("cloudFiles") \
                             .option("cloudFiles.format", "csv") \
                             .option("inferSchema", "true") \
                             .option("cloudFiles.schemaLocation", "dbfs:/FileStore/dataset/car_schema") \
                             .option("cloudFiles.schemaHints", "price float, mileage float, engV float, year int")\
                             .load("dbfs:/mnt/Ashish/*")
 
car_stream_data.display()

car,price,body,mileage,engV,engType,registration,year,model,drive,_rescued_data


In [0]:
car_nissan =  car_stream_data.filter(car_stream_data.car == "Nissan").display()

car,price,body,mileage,engV,engType,registration,year,model,drive,_rescued_data


In [0]:
car_group = car_stream_data.groupBy("car","year").avg("price").display()

In [0]:
from typing import Optional
 
@udf
def premium_cars(price) -> Optional[bool]:
  return price > 20000

In [0]:
car_stream_data.withColumn('premium', premium_cars('price'))\
               .select('car', 'price', 'year', 'premium')\
               .display()


In [0]:
import urllib
from pyspark.sql.functions import col
from pyspark.sql.types import BooleanType

In [0]:
@udf
def like_new(year) -> Optional[bool]:
  return year >= 2016


In [0]:
spark.udf.register('like_new', like_new)

Out[35]: <function __main__.like_new(year) -> Union[bool, NoneType]>

In [0]:

car_stream_data.createOrReplaceTempView('car_stream_table')

In [0]:
spark.sql('select car, body, year, like_new(year) from car_stream_table').display()